In [7]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

Using matplotlib backend: MacOSX


In [2]:
from __future__ import print_function
import tensorflow as tf

## Tensorboard可视化
我们使用了之前的线性回归的例子，来展示tensorboard 的用法。具体的说明可以参考官方文档，当然也可以参考中文版的。

我们可以对关心的节点添加summary 操作（如果该节点对应的tensor里面有多个元素，一般使用histogram_summary来看它们的分布。比如说权重矩阵。如果该节点对应的是一个标量，如loss值，那么我们就可以用summary.scalar）

在TensorFlow中，所有的操作只有当你执行，或者另一个操作依赖于它的输出时才会运行。我们刚才创建的这些summary节点都围绕着你的图像：没有任何操作依赖于它们的结果。因此，为了生成汇总信息，我们需要运行所有这些节点。我们可以使用tf.merge_all_summaries来将他们合并为一个操作。

然后你可以执行合并命令，它会依据特点步骤将所有数据生成一个序列化的Summary protobuf对象。最后，为了将汇总数据写入磁盘，需要将汇总的protobuf对象传递给tf.train.Summarywriter。

SummaryWriter 的构造函数中包含了参数 logdir。这个 logdir 非常重要，所有事件都会写到它所指的目录下。此外，SummaryWriter 中还包含了一个可选择的参数 GraphDef。如果输入了该参数，那么 TensorBoard 也会显示你的图像。

另外非常重要的一点，典型的 TensorFlow 计算图有数以千计的节点，为简单起见，我们为节点对应的变量名划定范围，这样就会产生一个层级结构。默认情况下， 只有顶层节点会显示（该处不明白）。

In [3]:
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

In [4]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s' % n_layer

    # for tensorflow version < 0.12
    if int((tf.__version__).split('.')[1]) < 12:
        with tf.name_scope(layer_name):
            with tf.name_scope('weights'):
                Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
                tf.histogram_summary(layer_name + '/weights', Weights)
            with tf.name_scope('biases'):
                biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
                tf.histogram_summary(layer_name + '/biases', biases)
            with tf.name_scope('Wx_plus_b'):
                Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            if activation_function is None:
                outputs = Wx_plus_b
            else:
                outputs = activation_function(Wx_plus_b, )
            tf.histogram_summary(layer_name + '/outputs', outputs)
    else:  
        with tf.name_scope(layer_name):
            with tf.name_scope('weights'):
                Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
                tf.summary.histogram(layer_name + '/weights', Weights)
            with tf.name_scope('biases'):
                biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
                tf.summary.histogram(layer_name + '/biases', biases)
            with tf.name_scope('Wx_plus_b'):
                Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            if activation_function is None:
                outputs = Wx_plus_b
            else:
                outputs = activation_function(Wx_plus_b, )
            tf.summary.histogram(layer_name + '/outputs', outputs)
        return outputs


In [5]:

with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)

# tensorflow 在版本0.12之后，一些函数发生了变化，所以针对不同的版本我们会使用不同的summary函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))
    if int((tf.__version__).split('.')[1]) < 12:
        tf.scalar_summary('loss', loss)
    else:   
        tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()
if int((tf.__version__).split('.')[1]) < 12:  
    merged = tf.merge_all_summaries()
else:  
    merged = tf.summary.merge_all()

#把summary内容放到文件夹"logs/"里 
if int((tf.__version__).split('.')[1]) < 12:
    writer = tf.train.SummaryWriter('logs/', sess.graph)
else: 
    writer = tf.summary.FileWriter("logs/", sess.graph)


if int((tf.__version__).split('.')[1]) < 12:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)



In [6]:
# 接着打开terminal，进入你存放的文件夹地址上一层，运行命令 tensorboard --logdir='logs/'
# 会返回一个地址，然后用浏览器打开这个地址，在 graph 标签栏下打开
for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        #每50次将summary统计一次
        result = sess.run(merged,
                          feed_dict={xs: x_data, ys: y_data})
        writer.add_summary(result, i)

接着打开terminal，进入你存放的文件夹地址上一层，运行命令 tensorboard --logdir='logs/'
会返回一个地址，然后用浏览器打开这个地址，在 graph 标签栏下打开。记得一定要选择装了tensorflow的python环境。
<img src="imgs/cmd_tensorboard.png">

打开浏览器输入地址看到的内容：
<img src="imgs/emp_tensorboard.png">
从图中我们可以很明显的看到变量的层级结构。layer1下的biais，output和weights